In [1]:
import numpy as np # linear algebra
import pandas as pd 
import nltk
import ast
from collections import Counter
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [2]:
movies =pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")
print(movies.shape)
print(credits.shape)

(4803, 20)
(4803, 4)


In [3]:
movies.head(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [4]:
credits.head(5)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [5]:
movies = movies.merge(credits, left_on='id', right_on='movie_id')


In [6]:
movies.head(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,spoken_languages,status,tagline,title_x,vote_average,vote_count,movie_id,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [7]:
movies.shape

(4803, 24)

In [8]:
movies = movies[['movie_id','original_title','overview','genres','keywords','cast','crew']]

In [9]:
movies.isnull().sum()

movie_id          0
original_title    0
overview          3
genres            0
keywords          0
cast              0
crew              0
dtype: int64

In [10]:
movies.dropna(inplace=True)

In [11]:
movies.duplicated().sum()

0

In [12]:
movies["genres"][95]

'[{"id": 12, "name": "Adventure"}, {"id": 18, "name": "Drama"}, {"id": 878, "name": "Science Fiction"}]'

In [13]:
movies["genres"][0][9]

'8'

In [14]:
def list_create_genres(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):  #converts string values to its own  data types
        if counter < 4:
            L.append(i['name'])
        counter+=1
    return L 

In [15]:
#list_create_genres(movies["genres"][0])

In [16]:
movies["genres"]=movies["genres"].apply(list_create_genres)

In [17]:
def list_create_keywords(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):  #converts string values to its own  data types
        if counter < 10:
            L.append(i['name'])
        counter+=1
    return L 


In [18]:
#list_create_keywords(movies["keywords"][11])

In [19]:
movies["keywords"]=movies["keywords"].apply(list_create_keywords)

In [20]:
def list_create_cast(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 5:
            L.append(i['name'])
        counter+=1
    return L 

In [21]:
movies['cast'] = movies['cast'].apply(list_create_cast)
movies.head()

,movie_id,original_title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weave...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux, R...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman, A...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton,...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [22]:
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L

In [23]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [24]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [25]:
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

In [26]:
# Function to process a single sentence
def process_overview(sentence):
    if not isinstance(sentence, str):  # Handle non-string values like NaN
        return set()
    # Tokenize and tag parts of speech
    words = word_tokenize(sentence)
    tagged_list = nltk.pos_tag(words)
    # Filter for specific POS tags and return distinct, lowercase words
    filtered_values = {item[0].lower() for item in tagged_list if item[1] in {"NN", "NNP", "JJ", "NNS"}}
    return filtered_values

# Apply the function to every row in the 'overview' column
movies["processed_overview"] = movies["overview"].apply(process_overview)

# Display the updated dataframe
movies.head()

,movie_id,original_title,overview,genres,keywords,cast,crew,processed_overview
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver, ...",[JamesCameron],"{orders, century, pandora, unique, moon, torn,..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley, Ste...",[GoreVerbinski],"{earth, swann, dead, barbossa, captain, will, ..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux, Ralp...",[SamMendes],"{message, sends, forces, sinister, m, deceit, ..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman, Anne...",[ChristopherNolan],"{dent, dark, crimes, reputation, batman, gotha..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton, Wi...",[AndrewStanton],"{exotic, survival, rests, captain, people, bar..."


In [27]:
# Flatten all words from sets in the column into a single list
all_words = [word for word_set in movies["processed_overview"] for word in word_set]

# Count the frequency of each word
word_counts = Counter(all_words)

# Filter unique words used more than once
words_more_than_once = {word for word, count in word_counts.items() if count > 1}

# Update the column with filtered sets joined as strings
movies["processed_overview"] = [
    list(word_set & words_more_than_once) for word_set in movies["processed_overview"]
]

In [26]:
movies

,movie_id,original_title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver, ...",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley, Ste...",[GoreVerbinski]
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux, Ralp...",[SamMendes]
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman, Anne...",[ChristopherNolan]
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton, Wi...",[AndrewStanton]
...,...,...,...,...,...,...,...
4798,9367,El Mariachi,El Mariachi just wants to play his guitar and ...,"[Action, Crime, Thriller]","[unitedstates–mexicobarrier, legs, arms, paper...","[CarlosGallardo, JaimedeHoyos, PeterMarquardt,...",[RobertRodriguez]
4799,72766,Newlyweds,A newlywed couple's honeymoon is upended by th...,"[Comedy, Romance]",[],"[EdwardBurns, KerryBishé, MarshaDietlein, Cait...",[EdwardBurns]
4800,231617,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...","[Comedy, Drama, Romance, TVMovie]","[date, loveatfirstsight, narration, investigat...","[EricMabius, KristinBooth, CrystalLowe, GeoffG...",[ScottSmith]
4801,126186,Shanghai Calling,When ambitious New York attorney Sam is sent t...,[],[],"[DanielHenney, ElizaCoupe, BillPaxton, AlanRuc...",[DanielHsia]


In [30]:
movies['tags'] = movies['processed_overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [31]:
movies

,movie_id,original_title,overview,genres,keywords,cast,crew,processed_overview,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver, ...",[JamesCameron],"[orders, century, pandora, unique, moon, torn,...","[orders, century, pandora, unique, moon, torn,..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley, Ste...",[GoreVerbinski],"[earth, dead, captain, will, edge, life, turne...","[earth, dead, captain, will, edge, life, turne..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux, Ralp...",[SamMendes],"[message, sends, forces, sinister, m, deceit, ...","[message, sends, forces, sinister, m, deceit, ..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman, Anne...",[ChristopherNolan],"[dent, dark, crimes, reputation, batman, gotha...","[dent, dark, crimes, reputation, batman, gotha..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton, Wi...",[AndrewStanton],"[exotic, survival, captain, people, military, ...","[exotic, survival, captain, people, military, ..."
...,...,...,...,...,...,...,...,...,...
4798,9367,El Mariachi,El Mariachi just wants to play his guitar and ...,"[Action, Crime, Thriller]","[unitedstates–mexicobarrier, legs, arms, paper...","[CarlosGallardo, JaimedeHoyos, PeterMarquardt,...",[RobertRodriguez],"[work, henchmen, family, mariachi, tradition, ...","[work, henchmen, family, mariachi, tradition, ..."
4799,72766,Newlyweds,A newlywed couple's honeymoon is upended by th...,"[Comedy, Romance]",[],"[EdwardBurns, KerryBishé, MarshaDietlein, Cait...",[EdwardBurns],"[newlywed, arrivals, couple, respective, honey...","[newlywed, arrivals, couple, respective, honey..."
4800,231617,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...","[Comedy, Drama, Romance, TVMovie]","[date, loveatfirstsight, narration, investigat...","[EricMabius, KristinBooth, CrystalLowe, GeoffG...",[ScottSmith],"[dead, crimes, postal, change, office, servant...","[dead, crimes, postal, change, office, servant..."
4801,126186,Shanghai Calling,When ambitious New York attorney Sam is sent t...,[],[],"[DanielHenney, ElizaCoupe, BillPaxton, AlanRuc...",[DanielHsia],"[sam, beautiful, assistant, clever, career, jo...","[sam, beautiful, assistant, clever, career, jo..."


In [32]:
new =movies[["original_title","tags"]]

In [33]:
new

,original_title,tags
0,Avatar,"[orders, century, pandora, unique, moon, torn,..."
1,Pirates of the Caribbean: At World's End,"[earth, dead, captain, will, edge, life, turne..."
2,Spectre,"[message, sends, forces, sinister, m, deceit, ..."
3,The Dark Knight Rises,"[dent, dark, crimes, reputation, batman, gotha..."
4,John Carter,"[exotic, survival, captain, people, military, ..."
...,...,...
4798,El Mariachi,"[work, henchmen, family, mariachi, tradition, ..."
4799,Newlyweds,"[newlywed, arrivals, couple, respective, honey..."
4800,"Signed, Sealed, Delivered","[dead, crimes, postal, change, office, servant..."
4801,Shanghai Calling,"[sam, beautiful, assistant, clever, career, jo..."


In [35]:
 new["tags"][11]

['dangerous',
 'quantum',
 'james',
 'mr.',
 'm',
 'white',
 'urge',
 'bond',
 'casino',
 'fights',
 'personal',
 'determination',
 'complex',
 'truth',
 'mission',
 'woman',
 'anyone',
 'adventures',
 'solace',
 'organization',
 'vesper',
 'Adventure',
 'Action',
 'Thriller',
 'Crime',
 'killing',
 'undercover',
 'secretagent',
 'britishsecretservice',
 'DanielCraig',
 'OlgaKurylenko',
 'MathieuAmalric',
 'JudiDench',
 'GiancarloGiannini',
 'MarcForster']

In [40]:
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer

# Step 1: Ensure 'tags' is a string column
new["tags"] = new["tags"].apply(lambda x: " ".join(x) if isinstance(x, list) else x)
new["tags"] = new["tags"].str.lower()

# Step 2: Flatten all words and count frequencies
all_words = " ".join(new["tags"].astype(str)).split()
word_counts = Counter(all_words)

# Step 3: Filter words that appear more than once
words_more_than_once = {word for word, count in word_counts.items() if count > 1}

# Step 4: Use the filtered vocabulary in CountVectorizer
cv = CountVectorizer(stop_words="english", vocabulary=words_more_than_once)
vector = cv.fit_transform(new["tags"]).toarray()

# Check results
print("Shape of vectorized array:", vector.shape)
print("Filtered Vocabulary:", cv.get_feature_names_out())


C:\Users\Mrinmoy\AppData\Local\Temp\ipykernel_9560\2031938562.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new["tags"] = new["tags"].apply(lambda x: " ".join(x) if isinstance(x, list) else x)
C:\Users\Mrinmoy\AppData\Local\Temp\ipykernel_9560\2031938562.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new["tags"] = new["tags"].str.lower()


Shape of vectorized array: (4800, 14797)
Filtered Vocabulary: ["'" "'50s" "'60s" ... '“' '”' '…']


In [41]:
word_counts

Counter({'orders': 18,
         'century': 68,
         'pandora': 6,
         'unique': 25,
         'moon': 24,
         'torn': 12,
         'marine': 16,
         '22nd': 4,
         'civilization': 11,
         'alien': 120,
         'mission': 150,
         'action': 1216,
         'adventure': 880,
         'fantasy': 437,
         'sciencefiction': 503,
         'cultureclash': 14,
         'future': 138,
         'spacewar': 2,
         'spacecolony': 6,
         'society': 49,
         'spacetravel': 22,
         'futuristic': 21,
         'romance': 960,
         'space': 71,
         'samworthington': 8,
         'zoesaldana': 15,
         'sigourneyweaver': 22,
         'stephenlang': 6,
         'michellerodriguez': 10,
         'jamescameron': 7,
         'earth': 126,
         'dead': 110,
         'captain': 63,
         'will': 23,
         'edge': 9,
         'life': 701,
         'turner': 9,
         'elizabeth': 20,
         'nothing': 66,
         'ocean': 34,
  

In [43]:
new["tags"][0]

'orders century pandora unique moon torn marine 22nd civilization alien mission action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien samworthington zoesaldana sigourneyweaver stephenlang michellerodriguez jamescameron'

In [44]:
similarity = cosine_similarity(vector)

In [45]:
similarity[0]

array([1.        , 0.10444659, 0.06060606, ..., 0.02823912, 0.03413944,
       0.        ])

In [46]:
new[new['original_title'] == 'The Lego Movie'].index[0]

744

In [48]:
def recommend(movie):
    # Find the index of the input movie
    index = movies[movies['original_title'] == movie].index[0]
    
    # Calculate similarity distances and sort them in descending order
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1])
    
    # Display the recommended movies and their row details
    for i in distances[1:10]:  # Exclude the input movie itself
        print(f"Recommended Movie: {movies.iloc[i[0]].original_title}")
        print(f"Row Details:\n{movies.iloc[i[0]]}\n")


In [49]:
recommend('The Jungle Book')

Recommended Movie: The Jungle Book 2
Row Details:
movie_id                                                          14873
original_title                                        The Jungle Book 2
overview              Mowgli, missing the jungle and his old friends...
genres                                   [Family, Animation, Adventure]
keywords              [river, musical, tiger, village, feralchild, s...
cast                  [JohnGoodman, HaleyJoelOsment, MaeWhitman, Con...
crew                                                   [SteveTrenbirth]
processed_overview    [old, mowgli, jungle, man, unaware, danger, fr...
tags                  [old, mowgli, jungle, man, unaware, danger, fr...
Name: 2116, dtype: object

Recommended Movie: Roar
Row Details:
movie_id                                                           2989
original_title                                                     Roar
overview              Roar follows a family who are attacked by vari...
genres                

In [51]:
pickle.dump(movies,open('movies.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))